In [258]:
import re
import string 
def clean_text(text):
    text=" "+text+" "
    text=text.replace("Äôs"," us")#change org appeal specific typing error
    text=text.replace("don‚Äôt"," don't")#change org appeal specific typing error
    puncts=string.punctuation
    for punct in puncts:
        if(punct!="'"):
            s=" "+punct+" "
            text=text.replace(punct,s)
    text=text.replace("ain't","are not")
    text=text.replace("cannot","can not")
    text=text.replace("havn't","have not")
    text=text.replace("n't"," not")
    text=re.sub(r'\s+',' ',text)
    text=re.sub(r'\S*@\S*\s?',' ',text)#emails remove
    text=re.sub(r'http\S+',' ',text)#hyperlinks 
    text=re.sub(r' \d+',' ',text)#numbers
    return text

In [270]:
import nltk
from nltk.corpus import wordnet, sentiwordnet

NEGATORS = ["ain't",'cannot',"can't","didn't","doesn't","don't","hadn't",'hardly',
            "hasn't","haven't","havn't","isn't",'lack','lacking','lacks','neither',
            'never','no','nobody','none','nor','not','nothing','nowhere',"mightn't",
            "mustn't","needn't","oughtn't","shouldn't","wasn't",'without',"wouldn't",
            'are not','can not', 'did not','does not','do not','had not', 'has not',
            'have not','is not','might not','must not','need not','ought not','should not',
            'was not','would not', "n't"]
NEGATORS.sort(key = lambda x: 1/len(x))

import spacy
from spacy.lang.en.stop_words import STOP_WORDS



In [280]:
#all node names
entity_keys=nodesID.values()

#gives unique nodes in a sentence from NewText
def uniqueentitylist(phrase,entity_keys):
    tokenizer = RegexpTokenizer("[\w']+") 
    phraseTokens=list()
    phraseTokens=tokenizer.tokenize(phrase)
    unique_entity_set=set()
    unique_entity_list=[t for t in phraseTokens if t in entity_keys ]
    unique_entity_set.update(unique_entity_list)
    return list(unique_entity_set)

#counts no of unique nodes in a sentence from NewText
def uniqueentitycount(phrase,entity_keys):
    return len(uniqueentitylist(phrase,entity_keys))

#gives pair of unique nodes per sentence 
def entity_pairs_in_sent(l):
    l.sort()
    pairs=list()
    for i in range(len(l)):
        x=list()
        y=list()
        for j in range(i+1,len(l)):
            x.append(l[i])
            y.append(l[j])
        pairs.extend(zip(x,y))
    return pairs

In [420]:
def get_pos(phrase_list,sentID,depAnn):
    tup_lists=list()
    for phrase in phrase_list:
        tup_list=list()
        for i in range(0,len(ann.sentence[sentID].token)):
            if depAnn.sentence[sentID].token[i].word in phrase:
                tup_list.append((depAnn.sentence[sentID].token[i].word,depAnn.sentence[sentID].token[i].pos))
        tup_lists.append(tup_list)
    return tup_lists

"""with CoreNLPClient(properties={'annotators': 'coref', 'coref.algorithm' : 'statistical'}, timeout=60000, memory='16G') as client:
    dp_ann = client.annotate(text)
    chains = dp_ann.corefChain"""
def toROOTparse(dependencyAnn,sentID,x):
    snt=dependencyAnn.sentence[sentID]
    dependencyParse=snt.basicDependencies
    dp=dependencyParse
    upwardParse=dict()
    for e in dp.edge:
        upwardParse[snt.token[e.target-1].word]=snt.token[e.source-1].word
        
    [i]=dp.root
    root_word=snt.token[i-1].word
    if(x=="Node_1"):
        refs=node1[sentID]
    elif(x=="Node_2"):
        refs=node2[sentID]
    elif(x=="Node_3"):
        refs=node3[sentID]
    elif(x=="Node_4"):
        refs=node4[sentID]
    elif(x=="Node_5"):
        refs=node5[sentID]
    elif(x=="Node_6"):
        refs=node6[sentID]
    phrasefortokens=list()
    for ref in refs:
        refTokens=tokenizer.tokenize(ref)
        anspath=list()
        for r in refTokens:
            refpath=list()
            x=r
            while x!=root_word:
                refpath.append(upwardParse[x])
                x=upwardParse[x]
            refpath.append(root_word)
            if(len(refpath)>len(anspath)):
                anspath=refpath
        phrasefortokens.append(anspath)
    return phrasefortokens

"""for a sentence, given two entities, we access how
the entity is referenced in the sentence and get a bridge of words connecting both
entities in the dependency parse--- a list all such bridge words for the no of references
of each entity ---a list of tuples with words and pos tags"""
def findDependencyPhrase(sentID,x,y,dp_ann):
    pthlists=list()
    paths_x=toROOTparse(dp_ann,sentID,x)#a list of paths for all references
    paths_y=toROOTparse(dp_ann,sentID,y)#a list of paths for all references
    for px in paths_x:#px is a list of words
        pth=list()
        for py in paths_y:#py is a list of words
            for xword in px:
                xword=xword.lower()
                for yword in py:
                    yword=yword.lower()
                    if(xword==yword):
                        key=yword
                        break
            for xword in px:
                if xword==key:
                    break
                else: 
                    pth.append(xword)
            for yword in py:
                if yword==key:
                    break
                else: 
                    pth.append(yword)
            pth.append(key)
        pthlists.append(pth)
    return get_pos(pthlists,sentID,dp_ann)
    

In [416]:
dp_ann=ann
sentID=2
x="Node_4"
y="Node_5"
ll=findDependencyPhrase(sentID,x,y,dp_ann)
ll

[[('started', 'VBD'),
  ('Nations', 'NNP'),
  ('SORBOJOYA', 'NN'),
  ('project', 'NN'),
  ('assure', 'VB')]]

In [418]:
def word_to_sentiment(word, pos=None):
    """
    Given a word and an optional part of speech tag,
    returns the average sentiment score for that word using senti_synsets.
    """
    if word in STOP_WORDS or word in NEGATORS:
        return 0

    if pos == None:
        synsets = list(sentiwordnet.senti_synsets(word))

    elif pos != None:
        def get_wordnet_pos(tag):
            """
            Translating Spacy's POS tags to WordNet's POS tags
            https://stackoverflow.com/questions/1833252/java-stanford-nlp-part-of-speech-label
            """
            if tag.startswith('JJ'): return wordnet.ADJ
            elif tag.startswith('VB'): return wordnet.VERB
            elif tag.startswith('NN'): return wordnet.NOUN
            elif tag.startswith('RB'): return wordnet.ADV
            else: return wordnet.NOUN

        wordnet_pos = get_wordnet_pos(pos)
        synsets = list(sentiwordnet.senti_synsets(word.lower(), wordnet_pos))

    score = 0
    for el in synsets:
        score += el.pos_score()
        score -= el.neg_score()

    if score == 0 and pos != None:
        return word_to_sentiment(word, pos=None)

    if len(synsets) == 0: return 0

    score /= len(synsets)
    return score


In [417]:
def list_to_sentiment(pos_tags):
    """
    Finds the average sentiment of a list of tuples of (word, pos) using word_to_sentiment()
    """
    score = 0
    words_with_sentiment = 0
    for tup in pos_tags:
        word, pos = tup[0], tup[1]
        if word_to_sentiment(word, pos) != 0:
            words_with_sentiment += 1
            score += word_to_sentiment(word.lower(), pos)
    if score == 0:
        return 0
    else:
        return score/words_with_sentiment

## Algorithm

In [260]:
#upload text file
path='/Volumes/Seagate Expansion Drive/Minnesota /sentiment analysis/gender sample 1'
with open(path,'r') as f:
    text=f.read()
    
text=clean_text(text)#cleaning

In [304]:
""""!pip install stanza
import stanza
!echo "Downloading CoreNLP..."
!wget "http://nlp.stanford.edu/software/stanford-corenlp-4.0.0.zip" -O corenlp.zip
!unzip corenlp.zip
!mv ./stanford-corenlp-4.0.0 ./corenlp

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = "./corenlp"
from stanza.server import CoreNLPClient"""
#coreference resolution
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
with CoreNLPClient(properties={'annotators': 'coref', 'coref.algorithm' : 'statistical'}, timeout=60000, memory='16G') as client:
    ann = client.annotate(text)
    mychains = list()
    chains = ann.corefChain
    for chain in chains:
        mychain = list()
        # Loop through every mention of this chain
        for mention in chain.mention:
            # Get the sentence in which this mention is located, and get the words which are part of this mention
            # (we can have more than one word, for example, a mention can be a pronoun like "he", but also a compound noun like "His wife Michelle")
            words_list = ann.sentence[mention.sentenceIndex].token[mention.beginIndex:mention.endIndex]
            #build a string out of the words of this mention
            ment_word = ' '.join([x.word for x in words_list])
            mychain.append(ment_word)
        mychains.append(mychain)

for chain in mychains:
    print(' <-> '.join(chain))

Starting server with command: java -Xmx16G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-4e1774436e664ab7.props -preload coref
women 's of our country <-> themselves
their <-> They <-> Middle aged women
them <-> every women of Bangladesh <-> their
us <-> our <-> our <-> our <-> we
their <-> still women
women <-> their
% women <-> the women <-> the women
our country <-> our country <-> Bangladesh <-> our country <-> Bangladesh <-> Bangladesh <-> Bangladesh <-> Bangladesh <-> Bangladesh <-> the country <-> Bangladesh
the women 's of Bangladesh <-> their <-> Women from different sectors
them <-> their
SORBOJOYA <-> SORBOJOYA <-> United Nations and other " SORBOJOYA
its <-> it
its mother <-> the mother
the baby <-> The baby
it <-> using sanitary napkin
our <-> Our <-> Our <-> we <-> we
their <-> Women of rural areas , Rohingya women , tribal women <-> They <-> They
this

In [306]:
ment_word

'Prostate cancer'

In [292]:
"""
Changes annotated text
"""


chains = ann.corefChain
for chain in chains:
    for mention in chain.mention:
        indexrange=range(mention.beginIndex,mention.endIndex)
        for v in indexrange:
            if(v==mention.beginIndex):
                if chain.chainID in nodesfromchainID["Node_1"]:
                    ann.sentence[mention.sentenceIndex].token[v].word=' Node_1 '
                elif chain.chainID in nodesfromchainID["Node_3"]:
                    ann.sentence[mention.sentenceIndex].token[v].word=' Node_3 '
                elif chain.chainID in nodesfromchainID["Node_4"]:
                    ann.sentence[mention.sentenceIndex].token[v].word=' Node_4 '
                elif chain.chainID in nodesfromchainID["Node_5"]:
                    ann.sentence[mention.sentenceIndex].token[v].word=' Node_5 '
                elif chain.chainID in nodesfromchainID["Node_6"]:
                    ann.sentence[mention.sentenceIndex].token[v].word=' Node_6 '
            else:
                ann.sentence[mention.sentenceIndex].token[v].word=''
sentences=list()
for sent in ann.sentence:
    words=list()
    for tok in sent.token:
        words.append(tok.word)
    sentence=' '.join(words)
#    print(sentence)
    sentences.append(sentence)
NewText=' '.join(sentences)
print(NewText)

have signed . Let ‚  Node_1  get to , ! Mahfuz Islam started this petition to  Node_5       " is a project to assure women health service in  Node_4  . The purpose of  Node_5  is to provide safe menstrual health service to teenagers , tribal , Rohingya women and women of rural areas . Women plays a vital role towards  Node_1  nation . Nine and  Node_4       is women which is more than half of  Node_4    . So , the progress of  Node_4   is not possible by keeping  Node_4     . That 's the reason  Node_1  need to focus on women 's health and need to keep  Node_1  women 's safe . In  Node_4   Node_3   are ashamed to discuss on  Node_3  menstrual health .  Node_3           are unaware of safe menstrual health service .  Node_3  use clothes instead of sanitary napkins . Clothes are very unhygienic and unfriendly to skin .  Node_3  do not have any idea about safe sanitation and menstrual health . Then women who have less earning or striving in  Node_3  day to day life ca not afford a costly 

In [273]:


#done manually, could use clustering in future to group chains referencing similar entities
nodesfromchainID=dict()
nodesfromchainID["Node_3"]=list()
nodesfromchainID["Node_3"].extend([64,129,162,40,73,169,142,175,119,58])
nodesfromchainID["Node_1"]=list()
nodesfromchainID["Node_1"].extend([68,185])
nodesfromchainID["Node_4"]=list()
nodesfromchainID["Node_4"].extend([109,30])
nodesfromchainID["Node_5"]=list()
nodesfromchainID["Node_5"].extend([145])
nodesfromchainID["Node_6"]=list()
nodesfromchainID["Node_6"].extend([120])



#hash map to replace all entities
nodesID=dict()
nodesID["us"]="Node_1"
nodesID["you"]="Node_2"
nodesID["woman"]="Node_3"
nodesID["Bangladesh"]="Node_4"
nodesID["Surbojoya"]="Node_5"
nodesID["baby"]="Node_6"



In [266]:
#import nltk
#nltk.download('punkt')
#from nltk.tokenize import sent_tokenize 
"""original text"""
org_text=text
sentences=sent_tokenize(text)

In [297]:
ann.text

'  have signed . Let‚ us get to  ,  ! Mahfuz Islam started this petition to United Nations and  other " SORBOJOYA " is a project to assure women health service in Bangladesh . The purpose of SORBOJOYA is to provide safe menstrual health service to teenagers , tribal , Rohingya women and women of rural areas . Women plays a vital role towards our nation . Nine and a half crore population of Bangladesh is women which is more than half of the total population . So , the progress of the country is not possible by keeping this huge population behind . That\'s the reason we need to focus on women\'s health and need to keep our women\'s safe . In Bangladesh still women are ashamed to discuss on their menstrual health . Women of rural areas , Rohingya women , tribal women are unaware of safe menstrual health service . They use clothes instead of sanitary napkins . Clothes are very unhygienic and unfriendly to skin . They do not have any idea about safe sanitation and menstrual health . Then wo

In [307]:
"""This is to store a dict of the reference string of an entity by sentence index"""
node2=dict()
node3=dict()
node4=dict()
node5=dict()
node6=dict()
node1=dict()
for i in range(0,len(sentences)):
    node3[i]=list()
    node4[i]=list()
    node5[i]=list()
    node6[i]=list()
    node1[i]=list()
    node2[i]=list()

for chain in chains:####text ###ann shouldnt be replaced
    if chain.chainID in nodesfromchainID["Node_3"]:
        for mention in chain.mention:
            words_list = ann.sentence[mention.sentenceIndex].token[mention.beginIndex:mention.endIndex]
            ment_word = ' '.join([x.word for x in words_list])
            node3[mention.sentenceIndex].append(ment_word)#word may contain some puncts
    elif chain.chainID in nodesfromchainID["Node_1"]:
        for mention in chain.mention:
            words_list = ann.sentence[mention.sentenceIndex].token[mention.beginIndex:mention.endIndex]
            ment_word = ' '.join([x.word for x in words_list])
            node1[mention.sentenceIndex].append(ment_word)#word may contain some puncts
    elif chain.chainID in nodesfromchainID["Node_4"]:
        for mention in chain.mention:
            words_list = ann.sentence[mention.sentenceIndex].token[mention.beginIndex:mention.endIndex]
            ment_word = ' '.join([x.word for x in words_list])
            node4[mention.sentenceIndex].append(ment_word)#word may contain some puncts
    elif chain.chainID in nodesfromchainID["Node_5"]:
        for mention in chain.mention:
            words_list = ann.sentence[mention.sentenceIndex].token[mention.beginIndex:mention.endIndex]
            ment_word = ' '.join([x.word for x in words_list])
            node5[mention.sentenceIndex].append(ment_word)#word may contain some puncts
    elif chain.chainID in nodesfromchainID["Node_6"]:
        for mention in chain.mention:
            words_list = ann.sentence[mention.sentenceIndex].token[mention.beginIndex:mention.endIndex]
            ment_word = ' '.join([x.word for x in words_list])
            node6[mention.sentenceIndex].append(ment_word)#word may contain some puncts
            
            
editedsentences=sent_tokenize(NewText)

""" Some words might represent the entities but may not be referenced
to include such words -- match the lemma of the word to the entity dictionary lemma"""
#second stage cleaning by lemma matching
with CoreNLPClient(annotators=['lemma'], memory='4G',endpoint='http://localhost:9003') as client:
    document = client.annotate(NewText)# since we don't want to include the words that are referenced already addressed
    for i, sent in enumerate(document.sentence):
        for t in sent.token:
            if (t.lemma=="we"):
                node1[i].append(t.word)
                editedsentences[i]=editedsentences[i].replace(t.word,"Node_1")
            elif (t.lemma=="you"):
                node2[i].append(t.word)
                editedsentences[i]=editedsentences[i].replace(t.word,"Node_2")
            elif (t.lemma=="woman"):
                node3[i].append(t.word)
                editedsentences[i]=editedsentences[i].replace(t.word,"Node_3")
            elif (t.lemma=="country"):
                node4[i].append(t.word)
                editedsentences[i]=editedsentences[i].replace(t.word,"Node_4")
                
NT=' '.join(editedsentences)
editedsentences=sent_tokenize(NT)
NewText=NT
text=org_text# to undo any changes on text

Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9003 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-298a7e62eae540e2.props -preload lemma


In [308]:
ment_word

'a half crore population of Bangladesh'

#### dependency parsing and POS tagging 

In [277]:
# parts of speech tagging and dependency parsing
with CoreNLPClient(annotators=['pos','depparse'], timeout=60000, memory='16G', endpoint='http://localhost:9002') as client:
    ann = client.annotate(text)
#    sentence=ann.sentence[2]
#    dependency_parse = sentence.basicDependencies

Starting server with command: java -Xmx16G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9002 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-3b02b4be99fe408b.props -preload pos,depparse


In [278]:
#result of dependency parsing of any sentence for ex here sentence 2
sentence=ann.sentence[2]
dp=sentence.basicDependencies
res=list()
for e in dp.edge:
    s=e.dep+"("+sentence.token[e.source-1].word+"-"+str(e.source)+","+sentence.token[e.target-1].word+"-"+str(e.target)+")"
    #print(e.dep,"(",sentence.token[e.source-1].word,", ",sentence.token[e.target-1].word,")")
    res.append(s)
print(res)
#parse is done



['case(Nations-8,to-6)', 'compound(Nations-8,United-7)', 'conj(Nations-8,SORBOJOYA-12)', 'cc(SORBOJOYA-12,and-9)', 'amod(SORBOJOYA-12,other-10)', 'punct(SORBOJOYA-12,"-11)', 'cop(project-16,is-14)', 'det(project-16,a-15)', 'acl(project-16,assure-18)', 'obl(assure-18,Bangladesh-23)', 'mark(assure-18,to-17)', 'obj(assure-18,service-21)', 'compound(service-21,women-19)', 'compound(service-21,health-20)', 'case(Bangladesh-23,in-22)', 'compound(Islam-2,Mahfuz-1)', 'obl(started-3,Nations-8)', 'punct(started-3,.-24)', 'punct(started-3,"-13)', 'dep(started-3,project-16)', 'nsubj(started-3,Islam-2)', 'obj(started-3,petition-5)', 'det(petition-5,this-4)']


#### sentiment scores entity pair wise

In [421]:
PairSentimentDict_netCount=dict()
PairSentimentDict_netCount[("Node_1","Node_2")]=0
PairSentimentDict_netCount[("Node_1","Node_3")]=0
PairSentimentDict_netCount[("Node_1","Node_4")]=0
PairSentimentDict_netCount[("Node_1","Node_5")]=0
PairSentimentDict_netCount[("Node_1","Node_6")]=0
PairSentimentDict_netCount[("Node_2","Node_3")]=0
PairSentimentDict_netCount[("Node_2","Node_4")]=0 
PairSentimentDict_netCount[("Node_2","Node_5")]=0
PairSentimentDict_netCount[("Node_2","Node_6")]=0
PairSentimentDict_netCount[("Node_3","Node_4")]=0
PairSentimentDict_netCount[("Node_3","Node_5")]=0
PairSentimentDict_netCount[("Node_3","Node_6")]=0
PairSentimentDict_netCount[("Node_4","Node_5")]=0
PairSentimentDict_netCount[("Node_5","Node_6")]=0
PairSentimentDict_netsentiment=dict()
PairSentimentDict_netsentiment[("Node_1","Node_2")]=0
PairSentimentDict_netsentiment[("Node_1","Node_3")]=0
PairSentimentDict_netsentiment[("Node_1","Node_4")]=0
PairSentimentDict_netsentiment[("Node_1","Node_5")]=0
PairSentimentDict_netsentiment[("Node_1","Node_6")]=0
PairSentimentDict_netsentiment[("Node_2","Node_3")]=0
PairSentimentDict_netsentiment[("Node_2","Node_4")]=0
PairSentimentDict_netsentiment[("Node_2","Node_5")]=0
PairSentimentDict_netsentiment[("Node_2","Node_6")]=0
PairSentimentDict_netsentiment[("Node_3","Node_4")]=0
PairSentimentDict_netsentiment[("Node_3","Node_5")]=0
PairSentimentDict_netsentiment[("Node_3","Node_6")]=0
PairSentimentDict_netsentiment[("Node_4","Node_5")]=0
PairSentimentDict_netsentiment[("Node_5","Node_6")]=0
#from nltk.tokenize import RegexpTokenizer 
#only words 
tokenizer = RegexpTokenizer("[\w']+") 
no_of_sentences=len(editedsentences)
#no_of_sentences
#sentences are sentence tokens
for sentID in range(0,no_of_sentences):
    print(sentID)
    if sentID!=7:# was hanging due to statement 7 so skipped it
        l=uniqueentitylist(editedsentences[sentID],entity_keys)
        if(len(l)==1 and l[0] != 'Node_1'):
            1
            print(l) 
        else:
            ent_pairs=entity_pairs_in_sent(l)
            for tup in ent_pairs:
                (x,y)=tup
                print(x,y)
                listofpairDepPhrase=findDependencyPhrase(sentID,x,y,ann)
                print(listofpairDepPhrase)
                for DepPhrase in listofpairDepPhrase:
                    print(DepPhrase)
                    PairSentimentDict_netsentiment[tup]=PairSentimentDict_netsentiment[tup]+list_to_sentiment(DepPhrase)
                    PairSentimentDict_netCount[tup]=PairSentimentDict_netCount[tup]+1

0
1
2
Node_3 Node_4
[]
Node_3 Node_5
[]
Node_4 Node_5
[[('started', 'VBD'), ('Nations', 'NNP'), ('SORBOJOYA', 'NN'), ('project', 'NN'), ('assure', 'VB')]]
[('started', 'VBD'), ('Nations', 'NNP'), ('SORBOJOYA', 'NN'), ('project', 'NN'), ('assure', 'VB')]
3
Node_3 Node_5
[]
4
Node_1 Node_3
[[]]
[]
5
Node_3 Node_4
[]
6
['Node_4']
7
8
Node_3 Node_4
[[('ashamed', 'JJ'), ('discuss', 'VB'), ('health', 'NN')], [('women', 'NNS'), ('ashamed', 'JJ')]]
[('ashamed', 'JJ'), ('discuss', 'VB'), ('health', 'NN')]
[('women', 'NNS'), ('ashamed', 'JJ')]
9
['Node_3']
10
['Node_3']
11
12
['Node_3']
13
['Node_3']
14
Node_3 Node_4
[[('women', 'NNS'), ('country', 'NN'), ('derive', 'VBP')], [('women', 'NNS'), ('country', 'NN'), ('derive', 'VBP')]]
[('women', 'NNS'), ('country', 'NN'), ('derive', 'VBP')]
[('women', 'NNS'), ('country', 'NN'), ('derive', 'VBP')]
15
['Node_3']
16
['Node_3']
17
Node_3 Node_4
[]
18
['Node_3']
19
['Node_3']
20
['Node_3']
21
['Node_3']
22
Node_3 Node_4
[[('growing', 'VBG'), ('country',

In [423]:
#sentiment scores


for tup in PairSentimentDict_netsentiment.keys():
    (x,y)=tup
    if PairSentimentDict_netsentiment[tup]!=0:
        s=PairSentimentDict_netsentiment[tup]
        n=PairSentimentDict_netCount[tup]
        pairwiseSentiment=s/n
        print(x,y,":",pairwiseSentiment)

Node_1 Node_3 : 0.07124068061568062
Node_1 Node_4 : 0.13228039321789323
Node_3 Node_4 : -0.10892030423280426
Node_3 Node_6 : 0.002751018170426065
Node_4 Node_5 : 0.06398809523809525


In [424]:
# count of number of pair occurences
PairSentimentDict_netCount

{('Node_1', 'Node_2'): 1,
 ('Node_1', 'Node_3'): 4,
 ('Node_1', 'Node_4'): 2,
 ('Node_1', 'Node_5'): 0,
 ('Node_1', 'Node_6'): 0,
 ('Node_2', 'Node_3'): 0,
 ('Node_2', 'Node_4'): 0,
 ('Node_2', 'Node_5'): 0,
 ('Node_2', 'Node_6'): 0,
 ('Node_3', 'Node_4'): 8,
 ('Node_3', 'Node_5'): 0,
 ('Node_3', 'Node_6'): 1,
 ('Node_4', 'Node_5'): 1,
 ('Node_5', 'Node_6'): 0}

In [427]:
#dict for meaning of entity key values

nodesID
#pair wise sentiment

{'us': 'Node_1',
 'you': 'Node_2',
 'woman': 'Node_3',
 'Bangladesh': 'Node_4',
 'Surbojoya': 'Node_5',
 'baby': 'Node_6'}